In [ ]:
import math
import matplotlib as mpl
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import sys

In [ ]:
sys.path.append('../code/')

In [ ]:
from utils import split_df, aggregate_df

In [ ]:
## Create a pandas dataframe to hold the gene expression data
GENE_EXP_DATA = "../data/raw_data/3_summary_rpkm.xls"
meta4_exp = pd.read_csv(GENE_EXP_DATA, sep='\t')
genomes = pd.read_csv('../data/genomes_curated.tsv', sep='\t')

In [ ]:
meta4_exp = meta4_exp.merge(genomes)
meta4_exp.head(3)

In [ ]:
split_by_type = split_df(meta4_exp, 'type')
split_by_type.keys()

In [ ]:
split_by_type['m'].shape

In [ ]:
split_by_type['nmm'].shape

In [ ]:
methanotroph_abundances = aggregate_df(split_by_type['m'], 'genome', colnorm=True)
methanotroph_abundances.shape

In [ ]:
methylotroph_abundances = aggregate_df(split_by_type['nmm'], 'genome', colnorm=True)
methylotroph_abundances.shape

In [ ]:
methanotroph_abundances.head()

In [ ]:
sample_info = pd.read_csv('../data/sample_info.tsv', sep='\t')
sample_info.head(3)

In [ ]:
def prepare_df(df, sample_info):
    df = df.copy()  # for dev purposes
    # 'LakWasM130_HOW14_2_rpkm' --> '130_HOW14'
    df.columns = df.columns.str.extract('([0-9]*_[A-z]*[0-9]*)_', expand=False)
    # merge on the sample info to parse the oxygen, rep, week info.
    # The sample info are columns.  Transpose and reset index to access them. 
    orgs_as_cols = df.T.reset_index()
    orgs_as_cols.rename(columns={'index':'ID'}, inplace=True)
    df = pd.merge(orgs_as_cols, sample_info)
    df.set_index(['ID', 'oxy', 'rep', 'week', 'project'], inplace=True)
    return df
    

In [ ]:
methane = prepare_df(methanotroph_abundances, sample_info)

In [ ]:
methane.head(2)

In [ ]:
sns.heatmap(methane, cmap="YlGnBu")

In [ ]:
methane.iloc[methane.index.get_level_values('oxy') == 'Low']

In [ ]:
plot_data = methane.iloc[(methane.index.get_level_values('oxy') == 'Low') & 
           (methane.index.get_level_values('rep') == 1)]

In [ ]:
plot_data = methane.iloc[(methane.index.get_level_values('oxy') == 'Low') & 
                   (methane.index.get_level_values('rep') == 1)].T
sns.heatmap(plot_data)

In [ ]:
methanol = prepare_df(methylotroph_abundances, sample_info)

In [ ]:
plot_data = methanol.iloc[(methanol.index.get_level_values('oxy') == 'Low') & 
                   (methanol.index.get_level_values('rep') == 1)].T
sns.heatmap(plot_data)